## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/efron


## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_gbdt_'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'efron'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6387052937179566
Integrated Brier Score: 0.16360450278853264
{'model': 'efron', 'dataset': 'METABRIC', 'cindex_train': [0.6359974840960699], 'cindex_test': [0.6387052937179566], 'ibs_train': [0.16526710566334415], 'ibs_test': [0.16360450278853264]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5334767140358988
Integrated Brier Score: 0.17692196536510052
{'model': 'efron', 'dataset': 'METABRIC', 'cindex_train': [0.6346963749740127], 'cindex_test': [0.5334767140358988], 'ibs_train': [0.16207946306553367], 'ibs_test': [0.17692196536510052]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6433484647253553
Integrated Brier Score: 0.16811951536760544
{'model': 'efron', 'dataset

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,efron,METABRIC,0.635997,0.638705,0.165267,0.163605
1,efron,METABRIC,0.634696,0.533477,0.162079,0.176922
2,efron,METABRIC,0.672508,0.643348,0.153729,0.168120
3,efron,METABRIC,0.646785,0.637412,0.154708,0.163867
4,efron,METABRIC,0.667950,0.632040,0.160380,0.171782
5,efron,FLCHAIN,0.794753,0.803535,0.098275,0.099229
6,efron,FLCHAIN,0.800746,0.774973,0.097504,0.102673
7,efron,FLCHAIN,0.743940,0.740108,0.105379,0.106900
8,efron,FLCHAIN,0.794667,0.800128,0.099897,0.096223
9,efron,FLCHAIN,0.800471,0.787344,0.097028,0.097744


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'efron'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5981630309988519
Integrated Brier Score: 0.19462780359793383
{'model': 'efron', 'dataset': 'BLCA', 'cindex_train': [0.6989448230380303], 'cindex_test': [0.5981630309988519], 'ibs_train': [0.2128746601139805], 'ibs_test': [0.19462780359793383]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5747527632344386
Integrated Brier Score: 0.23794136498812204
{'model': 'efron', 'dataset': 'BLCA', 'cindex_train': [0.9178959698378418], 'cindex_test': [0.5747527632344386], 'ibs_train': [0.128024593738516], 'ibs_test': [0.23794136498812204]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5938606847697757
Integrated Brier Score: 0.21719584541315196
{'model': 'efron', 'dataset': 'BLCA', 'cin

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,efron,BLCA,0.698945,0.598163,0.212875,0.194628
1,efron,BLCA,0.917896,0.574753,0.128025,0.237941
2,efron,BLCA,0.827786,0.593861,0.194702,0.217196
3,efron,BLCA,0.758883,0.558418,0.209970,0.212850
4,efron,BLCA,0.901330,0.593281,0.175762,0.199697
5,efron,BRCA,0.583171,0.520476,0.185424,0.186045
6,efron,BRCA,0.853874,0.528373,0.164899,0.211266
7,efron,BRCA,0.894512,0.622593,0.161425,0.230480
8,efron,BRCA,0.464917,0.336838,0.185877,0.196027
9,efron,BRCA,0.829130,0.513753,0.164648,0.198903
